Code was made with the help of OpenAI LLM, I do not claim I write this code.

In [1]:
import os
import qdrant_client
from transformers import AutoTokenizer, pipeline
import torch
import transformers


/opt/anaconda3/envs/engauge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
with open('env/llama_token.txt', 'r') as file:
    token = file.read().strip()

print(device)

cpu


/opt/anaconda3/envs/engauge/lib/python3.12/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [4]:
local_folder_path = '/Users/rodrigosandon/Documents/CMSC/473/UMIACSWiki/dataset/raw_html'

In [7]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(
    url="https://d62f1a9d-e53d-4230-9569-49373c644499.us-east4-0.gcp.cloud.qdrant.io",
    api_key="aMhZGwQxW8KBWMfFqRze9dfE17z3leX1w7hOMxMKR-ijIdOCQUW-GQ"  # Replace with your actual API key
)

client.create_collection(
    collection_name="umiacs_wiki",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)  # Adjust size based on your embedding model
)

True

In [9]:
from qdrant_client.models import PointStruct
import os

def embed_document(file_path):
    # dummy
    return [0.0] * 768 

for idx, file_name in enumerate(os.listdir(local_folder_path)):
    file_path = os.path.join(local_folder_path, file_name)
    vector = embed_document(file_path)
    payload = {"file_name": file_name}

    client.upsert(
        collection_name="umiacs_wiki",
        wait=True,
        points=[
            PointStruct(id=idx, vector=vector, payload=payload)
        ]
    )

In [3]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024,
    token=token
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    token=token
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

query_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=1024,
    device_map="auto",
)

def find_most_similar_docs(qdrant_client, query_embedding, top_k=5):
    """
    Find the most similar documents by comparing the query embedding with stored embeddings in Qdrant.
    Args:
        qdrant_client: Qdrant client
        query_embedding: The query embedding
        top_k: Number of top similar documents to retrieve
    Returns:
        List of document IDs corresponding to the most similar documents
    """
    search_result = qdrant_client.search(
        collection_name="your_collection_name",  # Replace with your collection name
        query_vector=query_embedding.tolist(),
        limit=top_k
    )
    return [hit.id for hit in search_result]

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.25s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [11]:
message = input("Enter your message: ")

print("Tokenizing the message...")
inputs = tokenizer(message, return_tensors="pt")
print("Generating query embedding...")
outputs = query_pipeline.model(**inputs, output_hidden_states=True)
query_embedding = outputs.hidden_states[-1].mean(dim=1).squeeze().cpu().detach().numpy()
print("Query embedding generated.")

print("Searching for most similar documents...")
relevant_docs = find_most_similar_docs(qdrant_client, query_embedding, top_k=5)
print(f"Found {len(relevant_docs)} similar documents.")

print("Concatenating retrieved documents with the message...")
context = " ".join([qdrant_client.get_document("umiacs_wiki", doc_id).payload["content"] for doc_id in relevant_docs])
full_message = context + "\n" + message
print("Full message prepared.")

print("Generating response...")
sequences = query_pipeline(
    full_message,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

question = sequences[0]['generated_text'][:len(message)]
answer = sequences[0]['generated_text'][len(message):]

print("Question:", question)
print("Answer:", answer)

RuntimeError: MPS backend out of memory (MPS allocated: 45.84 GB, other allocations: 3.73 MB, max allowed: 45.90 GB). Tried to allocate 224.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).